In [1]:
# all imports
import csv
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import progressbar
import sys
import zipfile
import glob, os
import unidecode
import unicodedata
#from tqdm.notebook import tqdm
import matplotlib.pyplot as plt
import seaborn as sns

%matplotlib inline

In [2]:
# change the initial folder and create ./output and ./output_adgroups in it manually
path = 'D:/RoiData/'

In [3]:
# check adgroups in groupby 

numeric_columns = ['conversions', 'netRevenue', 'grossProfit', 'deliveries',
       'timeGranularity', 'visits', 'entrances', 'bounces', 'hits',
       'pageViews', 'users', 'newUsers', 'impressions', 'marketingInvestment',
       'clicks', 'platformConversions', 'platformNetRevenue']


def mark_zero_rows(data):
    not_in_df_cols = len(set(numeric_columns) - set(list(data.columns)))
    if not_in_df_cols != 0:
        for col in not_in_df_cols:
            numeric_columns.remove(col)
    for idx in data[data[numeric_columns].sum(axis=1)==0].index:
        data.at[idx, 'deleted'] = True


def check_campaigns(df):
    campaigns = set(df['webCampaign_tr'].value_counts().index) & set(data['adCampaign_tr'].value_counts().index)
    #print(campaigns)
    
    for idx in list(df[(df['adCampaign_tr'].isin(campaigns)) & (df['webCampaign'].isnull())].index):
        data.at[idx, 'webCampaign'] = data.at[idx, 'adCampaign']
        data.at[idx, 'mapped'] = True
        #print(idx)
        
        
    for idx in list(df[(df['webCampaign_tr'].isin(campaigns)) & (df['adCampaign'].isnull())].index):
        data.at[idx, 'adCampaign'] = data.at[idx, 'webCampaign']
        data.at[idx, 'mapped'] = True
        
    return df

In [12]:
# control if there are 32 files (in initial folder in initial state)
files = [f for f in os.listdir(path) if os.path.isfile(os.path.join(path, f))]
len(files)

1

In [13]:
# read data
## process campaigns and save

for file in files:
    print(f'READING {file}...')
    # read data
    data = pd.read_csv(f'{path}{file}', delimiter='\t')
    
    print(f'Processing data...')
    # initial transformation = lowecase
    data['webCampaign_tr'] = data['webCampaign']
    data['adCampaign_tr'] = data['adCampaign']

#     data['webCampaign_tr'] =  data['webCampaign_tr'].fillna('')
#     data['adCampaign_tr'] =  data['adCampaign_tr'].fillna('')

    # add sanity check (value_counts are equal before and after) (!)
    data[data['webCampaign'].notnull()]['webCampaign_tr'] = data[data['webCampaign'].notnull()]['webCampaign_tr'].str.lower()
    data[data['adCampaign'].notnull()]['adCampaign_tr'] = data[data['adCampaign'].notnull()]['adCampaign_tr'].str.lower()

    # sanity check
    # add sanity check: if there are campaigns which are both upper and lower case and are different, that is not good!!! 

    # add information about transformation applied
    data['lowercase'] = True
    data['replace+'] = False
    data['diacritics'] = False
    data['mapped'] = False
    
    # prepare data before transformation
    d = data.groupby(['site', 'platformCode']).apply(check_campaigns)

    # apply transformation to adCampaign in case it was not mapped yet
    # replace + with space
    for idx in data[(~data['mapped']) & (data['adCampaign_tr'].notnull())].index:
        data.at[idx, 'adCampaign_tr'] = data.at[idx, 'adCampaign_tr'].replace('+', ' ')
        data.at[idx, 'replace+'] = True
    d = data.groupby(['site', 'platformCode']).apply(check_campaigns)


    # # delete diacritics
    for idx in data[(~data['mapped']) & (data['adCampaign_tr'].notnull())].index:
        data.at[idx, 'adCampaign_tr'] = unidecode.unidecode(data.at[idx, 'adCampaign_tr']) #.map(unidecode.unidecode)
        data.at[idx, 'diacritics'] = True
    d = data.groupby(['site', 'platformCode']).apply(check_campaigns)
    
    
    print(f'Saving data...')
    data.to_csv(f'{path}output/{file.split(".")[0]}_output.csv')
    print('=== DONE ===')
    

READING vodafone_test.csv...
Processing data...


C:\Users\tomas\Anaconda3\lib\site-packages\ipykernel_launcher.py:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
C:\Users\tomas\Anaconda3\lib\site-packages\ipykernel_launcher.py:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


Saving data...
=== DONE ===


In [14]:
## ADGROUPS

def check_adGroups(df):
    campaigns = set(df['webAdGroup_tr'].value_counts().index) & set(data['adGroup_tr'].value_counts().index)
    #print(campaigns)
    
    for idx in list(df[(df['adGroup_tr'].isin(campaigns)) & (df['webAdGroup'].isnull())].index):
        data.at[idx, 'webAdGroup'] = data.at[idx, 'adGroup']
        data.at[idx, 'mapped_group'] = True
        #print(idx)
        
        
    for idx in list(df[(df['webAdGroup_tr'].isin(campaigns)) & (df['adGroup'].isnull())].index):
        data.at[idx, 'adGroup'] = data.at[idx, 'webAdGroup']
        data.at[idx, 'mapped_group'] = True
        
    return df

In [16]:
# read data
files = [filename for root, dirs, files in os.walk(path) for filename in files if len(dirs)==0]

for file in files:
    print(f'READING {file}...')
    # read data
    data = pd.read_csv(f'{path}output/{file}')
        
    if 'webAdGroup' not in data.columns:
        print(f'{file} does not have webAdGroup.')
        continue
    if data[data['webAdGroup'].notnull()]['webAdGroup'].count() == 0:
        print(f'{file} have only empty adGroups.')
        continue
    
    print(f'Processing data...')
    # initial transformation = lowecase
    data['webAdGroup_tr'] = data['webAdGroup']
    data['adGroup_tr'] = data['adGroup']

#     data['webCampaign_tr'] =  data['webCampaign_tr'].fillna('')
#     data['adCampaign_tr'] =  data['adCampaign_tr'].fillna('')

    # add sanity check (value_counts are equal before and after) (!)
    data[data['webAdGroup'].notnull()]['webAdGroup_tr'] = data[data['webAdGroup_tr'].notnull()]['webAdGroup_tr'].str.lower()
    data[data['adGroup'].notnull()]['adGroup_tr'] = data[data['adGroup'].notnull()]['adGroup_tr'].str.lower()

    # sanity check
    # add sanity check: if there are campaigns which are both upper and lower case and are different, that is not good!!! 

    # add information about transformation applied
    data['lowercase_group'] = True
    data['replace+_group'] = False
    data['diacritics_group'] = False
    data['mapped_group'] = False
    
    # prepare data before transformation
    d = data.groupby(['site', 'platformCode', 'webCampaign']).apply(check_adGroups)

    # apply transformation to adCampaign in case it was not mapped yet
    # replace + with space
    for idx in data[(~data['mapped']) & (data['adGroup_tr'].notnull())].index:
        data.at[idx, 'adGroup_tr'] = data.at[idx, 'adGroup_tr'].replace('+', ' ')
        data.at[idx, 'replace+_group'] = True
    d = data.groupby(['site', 'platformCode']).apply(check_adGroups)


    # # delete diacritics
    for idx in data[(~data['mapped']) & (data['adGroup_tr'].notnull())].index:
        data.at[idx, 'adGroup_tr'] = unidecode.unidecode(data.at[idx, 'adGroup_tr']) #.map(unidecode.unidecode)
        data.at[idx, 'diacritics_group'] = True
    d = data.groupby(['site', 'platformCode', 'webCampaign']).apply(check_adGroups)
    
    
    print(f'Saving data...')
    data.to_csv(f'{path}output_adgroups/{file.split(".")[0]}_adgr.csv')
    print('=== DONE ===')


READING audiopartner_output.csv...
Processing data...


C:\Users\tomas\Anaconda3\lib\site-packages\ipykernel_launcher.py:25: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
C:\Users\tomas\Anaconda3\lib\site-packages\ipykernel_launcher.py:26: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


Saving data...
=== DONE ===
READING auriotest_output.csv...
auriotest_output.csv does not have webAdGroup.
READING axatest_output.csv...
Processing data...
Saving data...
=== DONE ===
READING bezvasport-test_output.csv...
Processing data...
Saving data...
=== DONE ===
READING bezvasporttest_output.csv...
Processing data...
Saving data...
=== DONE ===
READING bluestyle-test_output.csv...
Processing data...
Saving data...
=== DONE ===
READING brwtest_output.csv...
Processing data...
Saving data...
=== DONE ===
READING coolstufftest_output.csv...
Processing data...
Saving data...
=== DONE ===
READING czntest_output.csv...
czntest_output.csv does not have webAdGroup.
READING decodomatest_output.csv...
Processing data...
Saving data...
=== DONE ===
READING eashopisttest_output.csv...
eashopisttest_output.csv have only empty adGroups.
READING econeatest_output.csv...
econeatest_output.csv does not have webAdGroup.
READING emakotest_output.csv...
Processing data...
Saving data...
=== DONE ===

FileNotFoundError: [Errno 2] File b'D:/RoiData/output/audiopartner.csv' does not exist: b'D:/RoiData/output/audiopartner.csv'

# All other is not for production -- experiments

In [ ]:
# files = [filename for root, dirs, files in os.walk(path) for filename in files if len(dirs) == 0]
# #data1 = pd.read_csv(f'{path}output/{files[0].split(".")[0]}_output.csv')
# data2 = pd.read_csv(f'{path}output/{files[0].split(".")[0]}.csv')
# #data1.drop('Unnamed: 0', axis=1, inplace=True)
# data2.drop('Unnamed: 0', axis=1, inplace=True)



#for file in files: 
files = [f for f in os.listdir(os.path.join(path,'output_adgroups') if os.path.isfile(os.path.join(path, f))]

In [ ]:
# get columns to create dataframe
files = [filename for root, dirs, files in os.walk(path) for filename in files if len(dirs) == 0]
for k, file in tqdm(enumerate(files)): 
    data = pd.read_csv(f'{path}output/{file}')
    if k == 0:
        total_cols = set(data.columns)
    else: 
        total_cols = total_cols | set(data.columns)


In [ ]:
for k, file in enumerate(files): 
    print(f'processing file {file}')
    data = pd.read_csv(f'{path}output/{file}')
    for col in (total_cols - set(data.columns)):
        data[col] = ''
    data = data.reindex(columns=list(total_cols))
    if k == 0:
        data.to_csv(f'{path}output_adgroups/total.csv')
    else:
        data.to_csv(f'{path}output_adgroups/total.csv', mode='a', header=False)

{'Unnamed: 0',
 'adCampaign',
 'adCampaign_tr',
 'adGroup',
 'adMedium',
 'adSource',
 'attributionModel',
 'bounces',
 'channelCode',
 'channelName',
 'clicks',
 'conversionType',
 'conversions',
 'currencyCode',
 'dataPartitionCode',
 'dataSourceCode',
 'date',
 'deliveries',
 'deviceCategory',
 'diacritics',
 'entrances',
 'gaConversions',
 'gaNetRevenue',
 'grossProfit',
 'hits',
 'impressions',
 'investmentType',
 'lowercase',
 'mapped',
 'marketingInvestment',
 'netRevenue',
 'newUsers',
 'pageViews',
 'periodCode',
 'periodStartDate',
 'platformCode',
 'platformConversions',
 'platformName',
 'platformNetRevenue',
 'replace+',
 'salesChannel',
 'site',
 'sourcePlatform',
 'timeGranularity',
 'users',
 'visits',
 'webAdGroup',
 'webCampaign',
 'webCampaign_tr',
 'webGroup',
 'webMedium',
 'webSource'}

52

In [21]:
max_cols = a['hptronictest_output.csv']

for k, v in a.items():
    print(k, set(max_cols)-set(v))
    n = n | set(v)
    
print('------------------')    
for k, v in a.items():
    print(k, set(v)-set(max_cols))
    


audiopartner_output.csv {'gaConversions', 'adSource', 'gaNetRevenue', 'adMedium'}
auriotest_output.csv {'gaConversions', 'conversionType', 'gaNetRevenue', 'adSource', 'webAdGroup', 'adMedium'}
axatest_output.csv {'adSource', 'adMedium'}
bezvasport-test_output.csv {'conversionType', 'adSource', 'adMedium'}
bezvasporttest_output.csv {'conversionType', 'adSource', 'adMedium'}
bluestyle-test_output.csv {'adSource', 'adMedium'}
brwtest_output.csv {'adSource', 'adMedium'}
coolstufftest_output.csv {'adSource', 'adMedium'}
czntest_output.csv {'conversionType', 'webAdGroup', 'adSource', 'adMedium'}
decodomatest_output.csv {'adSource', 'adMedium'}
eashopisttest_output.csv set()
econeatest_output.csv {'conversionType', 'webAdGroup', 'adSource', 'adMedium'}
emakotest_output.csv {'adSource', 'adMedium'}
eyerimtest_output.csv {'webAdGroup', 'adSource', 'adMedium'}
factcool_output.csv {'adSource', 'adMedium'}
hptronictest_output.csv set()
icsteptest_output.csv {'adSource', 'adMedium'}
ketodiettest_ou

{'Unnamed: 0',
 'adCampaign',
 'adCampaign_tr',
 'adGroup',
 'adMedium',
 'adSource',
 'attributionModel',
 'bounces',
 'channelCode',
 'channelName',
 'clicks',
 'conversionType',
 'conversions',
 'currencyCode',
 'dataPartitionCode',
 'dataSourceCode',
 'date',
 'deliveries',
 'deviceCategory',
 'diacritics',
 'entrances',
 'gaConversions',
 'gaNetRevenue',
 'grossProfit',
 'hits',
 'impressions',
 'investmentType',
 'lowercase',
 'mapped',
 'marketingInvestment',
 'netRevenue',
 'newUsers',
 'pageViews',
 'periodCode',
 'periodStartDate',
 'platformCode',
 'platformConversions',
 'platformName',
 'platformNetRevenue',
 'replace+',
 'salesChannel',
 'site',
 'sourcePlatform',
 'timeGranularity',
 'users',
 'visits',
 'webAdGroup',
 'webCampaign',
 'webCampaign_tr',
 'webGroup',
 'webMedium',
 'webSource'}

In [15]:
a = set([1, 2, 3, 4])
b = set([2, 3, 4])
a-b

{1}

In [14]:
b-a

{5}

In [37]:
for i, file in enumerate(files):
    if i==0:
        data = pd.read_csv(f'{path}output/{file}')
    else: 
        data1 = pd.read_csv(f'{path}output/{file}')
        data = pd.concat([data, data1])
data.to_csv(f'{path}output/total.csv')

C:\Program Files (x86)\Python38-32\lib\site-packages\IPython\core\interactiveshell.py:3062: DtypeWarning: Columns (41) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


MemoryError: Unable to allocate 34.3 MiB for an array with shape (11, 408115) and data type int64

In [33]:
data1 = pd.concat(data[1])
    

{0:         Unnamed: 0        date       site currencyCode dataPartitionCode  \
 0                0  2020-10-24  kytary.cz          CZK          kytarycz   
 1                1  2020-10-24  kytary.cz          CZK          kytarycz   
 2                2  2020-10-24  kytary.pl          CZK          kytarypl   
 3                3  2020-10-24  kytary.ro          CZK          kytaryro   
 4                4  2020-10-24  kytary.fr          CZK          kytaryfr   
 ...            ...         ...        ...          ...               ...   
 131311      131311  2020-11-23  kytary.pl          CZK          kytarypl   
 131312      131312  2020-11-23  kytary.pl          CZK          kytarypl   
 131313      131313  2020-11-23  kytary.pl          CZK          kytarypl   
 131314      131314  2020-11-23  kytary.sk          CZK          kytarysk   
 131315      131315  2020-11-23  kytary.pl          CZK          kytarypl   
 
         periodCode attributionModel  \
 0         20201024            

,date,site,currencyCode,dataPartitionCode,periodCode,attributionModel,dataSourceCode,conversionType,salesChannel,conversions,...,channelCode,channelName,investmentType,periodStartDate,webCampaign_tr,adCampaign_tr,lowercase,replace+,diacritics,mapped
0,2020-10-24,kytary.cz,CZK,kytarycz,20201024,NaN,adsadwordsdaily_kytary.cz_adwords-campaign-wee...,NaN,online,0.0,...,googleadssearch,GoogleAds / Search,NaN,2020-10-24T00:00:00.0000000,NaN,(CZ) Klavesy - obecna - BROAD,True,True,True,False
1,2020-10-24,kytary.cz,CZK,kytarycz,20201024,NaN,adsadwordsdaily_kytary.cz_adwords-campaign-wee...,NaN,online,0.0,...,googleadspla,GoogleAds / PLA,NaN,2020-10-24T00:00:00.0000000,NaN,(CZ) PLA - CSS - obecná,True,False,False,True
2,2020-10-24,kytary.pl,CZK,kytarypl,20201024,NaN,gasessions_kytary.pl_ga-sessions-adgroups_8268,NaN,online,0.0,...,googleadspla,Google Ads / PLA,NaN,2020-10-24T00:00:00.0000000,(A) PLA - výrobce,NaN,True,False,False,True
3,2020-10-24,kytary.ro,CZK,kytaryro,20201024,NaN,gasessions_kytary.ro_ga-sessions-adgroups_8272,NaN,online,0.0,...,NaN,NaN,NaN,2020-10-24T00:00:00.0000000,yot_popup_success_opt-in2,NaN,True,False,False,False
4,2020-10-24,kytary.fr,CZK,kytaryfr,20201024,NaN,adsadwordsdaily_kytary.fr_adwords-campaign-dai...,NaN,online,0.0,...,googleadsother,Google Ads / Other,NaN,2020-10-24T00:00:00.0000000,NaN,(A) Brand,True,False,False,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
131311,2020-11-23,kytary.pl,CZK,kytarypl,20201123,NaN,gasessions_kytary.pl_ga-sessions-adgroups_8268,NaN,online,0.0,...,email,Email,NaN,2020-11-23T00:00:00.0000000,yot_flow_mail__pondelnisleva-a_flowpl,NaN,True,False,False,True
131312,2020-11-23,kytary.pl,CZK,kytarypl,20201123,NaN,gasessions_kytary.pl_ga-sessions-adgroups_8268,NaN,online,0.0,...,googleadsother,Google Ads / Other,NaN,2020-11-23T00:00:00.0000000,(A) Bicí - Obecná,NaN,True,False,False,True
131313,2020-11-23,kytary.pl,CZK,kytarypl,20201123,NaN,gasessions_kytary.pl_ga-sessions-adgroups_8268,NaN,online,0.0,...,googleadsother,Google Ads / Other,NaN,2020-11-23T00:00:00.0000000,(A) Fender - modely,NaN,True,False,False,True
131314,2020-11-23,kytary.sk,CZK,kytarysk,20201123,NaN,adsadwordsdaily_kytary.sk_adwords-campaign-wee...,NaN,online,0.0,...,googleadsroiminer,Google Ads/Roiminer,NaN,2020-11-23T00:00:00.0000000,NaN,ROIminer - S - olam,True,True,True,False


In [133]:
files = [filename for root, dirs, files in os.walk(f'{path}output/') for filename in files]
outputs = {}

for file in tqdm(files):
    print(f'READING {file}...')
    outputs[file] = {}
    # read data
    data = pd.read_csv(f'{path}output/{files[0]}')
    print('Done')
    data['deleted'] = False
    outputs[file]['adCampaigns'] = pivot_table(data, col_name='adCampaign', what='after', plot_me=False)
    outputs[file]['webCampaigns'] = pivot_table(data, col_name='webCampaign', what='after', plot_me=False)


READING audiopartner_output.csv...
Done
READING auriotest_output.csv...
Done
READING axatest_output.csv...
Done
READING bezvasport-test_output.csv...
Done
READING bezvasporttest_output.csv...
Done
READING bluestyle-test_output.csv...
Done
READING brwtest_output.csv...
Done
READING coolstufftest_output.csv...
Done
READING czntest_output.csv...
Done
READING decodomatest_output.csv...
Done
READING eashopisttest_output.csv...
Done
READING econeatest_output.csv...
Done
READING emakotest_output.csv...
Done
READING eyerimtest_output.csv...
Done
READING factcool_output.csv...
Done
READING hptronictest_output.csv...
Done
READING icsteptest_output.csv...
Done
READING ketodiettest_output.csv...
Done
READING muzikertest_output.csv...
Done
READING nctest_output.csv...
Done
READING packwaytest_output.csv...
Done
READING puraviatest--Infinity_output.csv...
Done
READING puraviatest_output.csv...
Done
READING spektretest_output.csv...
Done
READING spravcetest_output.csv...
Done
READING superzootest_out

In [131]:
outputs['audiopartner_output.csv']['adCampaign']

site,kytary.at,kytary.co.uk,kytary.cz,kytary.de,kytary.fr,kytary.hu,kytary.it,kytary.pl,kytary.ro,kytary.sk
platformCode,,,,,,,,,,
(direct),,,,,,,,,,
(not tracked),,,,,,,,,,
AdWords,100,20.4,10.1,87.5,100,11.3,100,36,14.2,6.5
Adform,,,33.3,,,,,,,
Adminis,,100,100,100,,100,,100,100,100
Arukereso,,,,,,95.9,,,,
Ceneo,,,,,,,,50,,
CjAffiliate,,,100,,,,,100,,100
Compari,,,,,,,,,97.9,


In [129]:
outputs['audiopartner_output.csv']['webCampaigns']

site,kytary.at,kytary.co.uk,kytary.cz,kytary.de,kytary.fr,kytary.hu,kytary.it,kytary.pl,kytary.ro,kytary.sk
platformCode,,,,,,,,,,
(direct),,,,,,,,,,
(not tracked),,,,,,,,,,
AdWords,100,100,100,100,100,100,100,100,100,76.9
Adform,,,100,,,,,,,
Adminis,,100,100,100,,100,,100,100,100
Arukereso,,,,,,100,,,,
Ceneo,,,,,,,,100,,
CjAffiliate,,,100,,,,,100,,100
Compari,,,,,,,,,100,


In [70]:
# GENERATE STATISTICS

# statistics % per platform
stats = pd.DataFrame(columns=['site', 'platformCode', 'transformationUsed', 'campaignMapped [%]', 'visitsMapped [%]', 'clicksMapped [%]'])

mark_zero_rows(data)





In [71]:
data

,Unnamed: 0,date,site,currencyCode,dataPartitionCode,periodCode,attributionModel,dataSourceCode,conversionType,salesChannel,...,channelName,investmentType,periodStartDate,webCampaign_tr,adCampaign_tr,lowercase,replace+,diacritics,mapped,deleted
0,0,2020-10-24,kytary.cz,CZK,kytarycz,20201024,NaN,adsadwordsdaily_kytary.cz_adwords-campaign-wee...,NaN,online,...,GoogleAds / Search,NaN,2020-10-24T00:00:00.0000000,NaN,(CZ) Klavesy - obecna - BROAD,True,True,True,False,True
1,1,2020-10-24,kytary.cz,CZK,kytarycz,20201024,NaN,adsadwordsdaily_kytary.cz_adwords-campaign-wee...,NaN,online,...,GoogleAds / PLA,NaN,2020-10-24T00:00:00.0000000,NaN,(CZ) PLA - CSS - obecná,True,False,False,True,False
2,2,2020-10-24,kytary.pl,CZK,kytarypl,20201024,NaN,gasessions_kytary.pl_ga-sessions-adgroups_8268,NaN,online,...,Google Ads / PLA,NaN,2020-10-24T00:00:00.0000000,(A) PLA - výrobce,NaN,True,False,False,True,False
3,3,2020-10-24,kytary.ro,CZK,kytaryro,20201024,NaN,gasessions_kytary.ro_ga-sessions-adgroups_8272,NaN,online,...,NaN,NaN,2020-10-24T00:00:00.0000000,yot_popup_success_opt-in2,NaN,True,False,False,False,False
4,4,2020-10-24,kytary.fr,CZK,kytaryfr,20201024,NaN,adsadwordsdaily_kytary.fr_adwords-campaign-dai...,NaN,online,...,Google Ads / Other,NaN,2020-10-24T00:00:00.0000000,NaN,(A) Brand,True,False,False,True,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
131311,131311,2020-11-23,kytary.pl,CZK,kytarypl,20201123,NaN,gasessions_kytary.pl_ga-sessions-adgroups_8268,NaN,online,...,Email,NaN,2020-11-23T00:00:00.0000000,yot_flow_mail__pondelnisleva-a_flowpl,NaN,True,False,False,True,False
131312,131312,2020-11-23,kytary.pl,CZK,kytarypl,20201123,NaN,gasessions_kytary.pl_ga-sessions-adgroups_8268,NaN,online,...,Google Ads / Other,NaN,2020-11-23T00:00:00.0000000,(A) Bicí - Obecná,NaN,True,False,False,True,False
131313,131313,2020-11-23,kytary.pl,CZK,kytarypl,20201123,NaN,gasessions_kytary.pl_ga-sessions-adgroups_8268,NaN,online,...,Google Ads / Other,NaN,2020-11-23T00:00:00.0000000,(A) Fender - modely,NaN,True,False,False,True,False
131314,131314,2020-11-23,kytary.sk,CZK,kytarysk,20201123,NaN,adsadwordsdaily_kytary.sk_adwords-campaign-wee...,NaN,online,...,Google Ads/Roiminer,NaN,2020-11-23T00:00:00.0000000,NaN,ROIminer - S - olam,True,True,True,False,False


In [73]:
wc_all = data[~data['deleted']].groupby(['site', 'platformCode'])['webCampaign'].nunique().to_frame()
wc_mapped_after = data[(~data['deleted']) & data['mapped']].groupby(['site', 'platformCode'])['webCampaign'].nunique().to_frame()
pd.concat([wc_all, wc_mapped_after/wc_all], axis=1)


webCampaign  webCampaign
site      platformCode                                        
kytary.at (not tracked)                         1          NaN
          AdWords                               6     1.000000
          DoubleClick                           2          NaN
          FacebookBusinessAdsManager            3     1.000000
          Organic                               0          NaN
...                                           ...          ...
kytary.sk HeurekaSk                            59     0.559322
          Internal                              0          NaN
          Organic                               0          NaN
          Samba                                19     0.736842
          Yottly                                6     0.833333

[81 rows x 2 columns]

In [65]:
wc_mapped_after

campaignMapped [%]
site         platformCode                                  
kytary.at    AdWords                                      6
             FacebookBusinessAdsManager                   3
kytary.co.uk AdWords                                     10
             Adminis                                      2
             DoubleClick                                  3
             FacebookBusinessAdsManager                   5
             Samba                                       14
kytary.cz    AdWords                                     48
             Adform                                       1
             Adminis                                     10
             CjAffiliate                                  7
             DogNet                                       9
             FacebookBusinessAdsManager                  12
             HeurekaCz                                   81
             Internal                                    30
             OneSignal                                    1
             Samba                                      100
             Sklik                                       49
             Yottly                                      14
             ZboziCz                                      1
kytary.de    AdWords                                      7
             Adminis                                      1
             FacebookBusinessAdsManager                   3
kytary.fr    AdWords                                      5
             FacebookBusinessAdsManager                   3
kytary.hu    AdWords                                     16
             Adminis                                      3
             Arukereso                                   70
             FacebookBusinessAdsManager                   8
             Yottly                                       2
kytary.it    AdWords                                      6
             FacebookBusinessAdsManager                   4
kytary.pl    AdWords                                     27
             Adminis                                      3
             Ceneo                                        1
             CjAffiliate                                  3
             DoubleClick                                  1
             FacebookBusinessAdsManager                   5
             Silverpop                                   20
             Yottly                                       2
kytary.ro    AdWords                                     15
             Adminis                                      2
             Compari                                     47
             DoubleClick                                  2
             FacebookBusinessAdsManager                   5
             Yottly                                       9
kytary.sk    AdWords                                     10
             Adminis                                      1
             CjAffiliate                                  7
             FacebookBusinessAdsManager                   7
             HeurekaSk                                   33
             Samba                                       14
             Yottly                                       5

In [74]:
wc_all = data[~data['deleted']].groupby(['site', 'platformCode'])['webCampaign'].nunique().to_frame()
wc_mapped_after = data[(~data['deleted']) & data['mapped']].groupby(['site', 'platformCode'])['webCampaign'].nunique().to_frame()
wc_mapped_before = data[(~data['deleted']) & data['mapped'] & (~data['replace+'] & ~data['diacritics'])].groupby(['site', 'platformCode'])['webCampaign'].nunique().to_frame()
wwc = pd.concat([wc_all, (wc_mapped_after/wc_all)*100,  (wc_mapped_before/wc_all)*100], axis=1)
wwc.fillna(0)

webCampaign  webCampaign  webCampaign
site      platformCode                                                     
kytary.at (not tracked)                         1     0.000000     0.000000
          AdWords                               6   100.000000   100.000000
          DoubleClick                           2     0.000000     0.000000
          FacebookBusinessAdsManager            3   100.000000   100.000000
          Organic                               0     0.000000     0.000000
...                                           ...          ...          ...
kytary.sk HeurekaSk                            59    55.932203    55.932203
          Internal                              0     0.000000     0.000000
          Organic                               0     0.000000     0.000000
          Samba                                19    73.684211    73.684211
          Yottly                                6    83.333333    83.333333

[81 rows x 3 columns]

In [122]:
def pivot_table(data, col_name='webCampaign', what='after', plot_me=False):
    wc_all = data[~data['deleted']].groupby(['site', 'platformCode'])[col_name].nunique().to_frame()
    if what=='after':
        wc_mapped_tr = data[(~data['deleted']) & data['mapped']].groupby(['site', 'platformCode'])[col_name].nunique().to_frame()
        
    if what=='before':
        wc_mapped_tr = data[(~data['deleted']) & data['mapped'] & (~data['replace+'] & ~data['diacritics'])].groupby(['site', 'platformCode'])[col_name].nunique().to_frame()

    
    a = ((wc_mapped_tr/wc_all)*100).round(1)
    a = a.reset_index(level='site')
    a = a.reset_index()
    a = a.pivot('platformCode', 'site', col_name)
    if not plot_me:
        a = a.fillna('')
    return a

In [123]:

#sns.heatmap(pivot_table(data, col_name='adCampaign', what='after'))
#sns.heatmap(pivot_table(data, col_name='webCampaign', what='after'))

site,kytary.at,kytary.co.uk,kytary.cz,kytary.de,kytary.fr,kytary.hu,kytary.it,kytary.pl,kytary.ro,kytary.sk
platformCode,,,,,,,,,,
(direct),,,,,,,,,,
(not tracked),,,,,,,,,,
AdWords,100,47.6,25.7,100,100,33.3,100,65.9,39.5,18.5
Adform,,,33.3,,,,,,,
Adminis,,100,100,100,,100,,100,100,100
Arukereso,,,,,,95.9,,,,
Ceneo,,,,,,,,50,,
CjAffiliate,,,100,,,,,100,,100
Compari,,,,,,,,,97.9,


uniqueWebCampaigns
site      platformCode                                  
kytary.at (not tracked)                                1
          AdWords                                      6
          DoubleClick                                  2
          FacebookBusinessAdsManager                   3
          Organic                                      0
...                                                  ...
kytary.sk HeurekaSk                                   59
          Internal                                     0
          Organic                                      0
          Samba                                       19
          Yottly                                       6

[81 rows x 1 columns]

In [56]:

wwc

webCampaign  webCampaign
site      platformCode                                        
kytary.at (not tracked)                         1          NaN
          AdWords                               6     1.000000
          DoubleClick                           2          NaN
          FacebookBusinessAdsManager            3     1.000000
          Organic                               0          NaN
...                                           ...          ...
kytary.sk HeurekaSk                            59     0.559322
          Internal                              0          NaN
          Organic                               0          NaN
          Samba                                19     0.736842
          Yottly                                6     0.833333

[81 rows x 2 columns]

In [307]:
 # initial transformation = lowecase
data['webCampaign_tr'] = data['webCampaign']
data['adCampaign_tr'] = data['adCampaign']

# add sanity check (value_counts are equal before and after) (!)
data[~data.isnull()]['webCampaign_tr'] = data[~data.isna()]['webCampaign_tr'].str.lower()
data[~data.isnull()]['adCampaign_tr'] = data[~data.isna()]['adCampaign_tr'].str.lower()

# sanity check
# add sanity check: if there are campaigns which are both upper and lower case and are different, that is not good!!! 

# add information about transformation applied
data['lowercase'] = 1
data['replace+'] = 0
data['diacritics'] = 0
data['mapped'] = 0

0         0
1         0
2         0
3         0
4         0
         ..
131311    0
131312    0
131313    0
131314    0
131315    0
Name: mapped, Length: 131316, dtype: int64

0         0
3         0
9         0
17        0
18        0
         ..
131298    0
131299    0
131303    0
131309    0
131314    0
Name: mapped, Length: 36309, dtype: int64

In [320]:
# prepare data before transformation
d = data.groupby(['site', 'platformCode']).apply(check_campaigns)

# apply transformation to adCampaign in case it was not mapped yet
# replace + with space
for idx in data[(data['mapped']==0) & (data['adCampaign_tr'].notnull())].index:
    data.at[idx, 'adCampaign_tr'] = data.at[idx, 'adCampaign_tr'].replace('+', ' ')
    data.at[idx, 'replace+'] = 1
d = data.groupby(['site', 'platformCode']).apply(check_campaigns)


# # delete diacritics
for idx in data[(data['mapped']==0) & (data['adCampaign_tr'].notnull())].index:
    data.at[idx, 'adCampaign_tr'] = unidecode.unidecode(data.at[idx, 'adCampaign_tr']) #.map(unidecode.unidecode)
    data.at[idx, 'diacritics'] = 1
d = data.groupby(['site', 'platformCode']).apply(check_campaigns)

In [339]:
# GENERATE STATISTICS
# pocet kampani web/ad per platform
wc_all = data.groupby(['site','platformCode'])['webCampaign'].nunique()
wc_mapped = data[data['mapped']==1].groupby(['site','platformCode'])['webCampaign'].nunique()

wa_all = data.groupby(['site','platformCode'])['adCampaign'].nunique()
wa_mapped = data[data['mapped']==1].groupby(['site','platformCode'])['adCampaign'].nunique()

# visits and clicks
wc_visits = data.groupby(['site','platformCode'])['webCampaign'].nunique()
campaigns_num = data_per_platform.groupby(col_name).agg({numeric:sum})
campaigns_proc = (campaigns_num/campaigns_num.sum(axis=0))*100

In [335]:
b

site          platformCode              
kytary.at     AdWords                         6
              FacebookBusinessAdsManager      3
kytary.co.uk  AdWords                        10
              Adminis                         2
              DoubleClick                     3
              FacebookBusinessAdsManager      5
              Samba                          14
kytary.cz     AdWords                        48
              Adform                          1
              Adminis                        10
              CjAffiliate                     7
              DogNet                          9
              FacebookBusinessAdsManager     12
              HeurekaCz                      81
              Internal                       30
              OneSignal                       1
              Samba                         100
              Sklik                          49
              Yottly                         14
              ZboziCz                         1

In [254]:
data1 = data.copy()

def xxx(df):
    #for idx in df.index:
    #    data1.at[idx, 'mapped'] = 1
    df['mapped'] = 1
    return df

data1['mapped'] = 0
dataagg = data1.groupby(['site']).apply(xxx)

dataagg['mapped']

0         1
1         1
2         1
3         1
4         1
         ..
131311    1
131312    1
131313    1
131314    1
131315    1
Name: mapped, Length: 131316, dtype: int64

In [242]:
data[data['webCampaign']==data['adCampaign']]

,date,site,currencyCode,dataPartitionCode,periodCode,attributionModel,dataSourceCode,conversionType,salesChannel,conversions,...,platformCode,platformName,adCampaign,adGroup,sourcePlatform,deviceCategory,channelCode,channelName,investmentType,periodStartDate
1773,2020-10-24,kytary.ro,CZK,kytaryro,20201024,NaN,omsorders_kytary.ro_oms_1427,NaN,online,3.0,...,(not tracked),(not tracked),(not tracked),(not tracked),(not tracked),(not tracked),(not tracked),(not tracked),NaN,2020-10-24T00:00:00.0000000
1840,2020-10-24,kytary.sk,CZK,kytarysk,20201024,NaN,omsorders_kytary.sk_oms_157,NaN,online,4.0,...,(not tracked),(not tracked),(not tracked),(not tracked),(not tracked),(not tracked),(not tracked),(not tracked),NaN,2020-10-24T00:00:00.0000000
2035,2020-10-24,kytary.cz,CZK,kytarycz,20201024,NaN,omsorders_kytary.cz_oms_156,NaN,online,50.0,...,(not tracked),(not tracked),(not tracked),(not tracked),(not tracked),(not tracked),(not tracked),(not tracked),NaN,2020-10-24T00:00:00.0000000
2271,2020-10-24,kytary.at,CZK,kytaryat,20201024,NaN,omsorders_kytary.at_oms_4698,NaN,online,2.0,...,(not tracked),(not tracked),(not tracked),(not tracked),(not tracked),(not tracked),(not tracked),(not tracked),NaN,2020-10-24T00:00:00.0000000
2520,2020-10-24,kytary.it,CZK,kytaryit,20201024,NaN,omsorders_kytary.it_oms_4699,NaN,online,3.0,...,(not tracked),(not tracked),(not tracked),(not tracked),(not tracked),(not tracked),(not tracked),(not tracked),NaN,2020-10-24T00:00:00.0000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
130743,2020-11-23,kytary.de,CZK,kytaryde,20201123,NaN,omsorders_kytary.de_oms_3283,NaN,online,2.0,...,(not tracked),(not tracked),(not tracked),(not tracked),(not tracked),(not tracked),(not tracked),(not tracked),NaN,2020-11-23T00:00:00.0000000
130871,2020-11-23,kytary.pl,CZK,kytarypl,20201123,NaN,omsorders_kytary.pl_oms_227,NaN,call,7.0,...,(not tracked),(not tracked),(not tracked),(not tracked),(not tracked),(not tracked),(not tracked),(not tracked),NaN,2020-11-23T00:00:00.0000000
130879,2020-11-23,kytary.fr,CZK,kytaryfr,20201123,NaN,omsorders_kytary.fr_oms_7478,NaN,online,3.0,...,(not tracked),(not tracked),(not tracked),(not tracked),(not tracked),(not tracked),(not tracked),(not tracked),NaN,2020-11-23T00:00:00.0000000
131231,2020-11-23,kytary.pl,CZK,kytarypl,20201123,NaN,omsorders_kytary.pl_oms_227,NaN,online,15.0,...,(not tracked),(not tracked),(not tracked),(not tracked),(not tracked),(not tracked),(not tracked),(not tracked),NaN,2020-11-23T00:00:00.0000000


In [243]:
data1 = data.copy()

ROIminer                        2601
DSA - produkty                  2208
ROIminer - S - cin              1274
(A) PLA - výrobce                937
ROIminer - S - olam              791
                                ... 
ElektronickeLicence                1
pl-pl.facebook.com                 1
yot_test_NL_kupon2_rijen_ro        1
ROIminer - S - perus               1
yot_email_rolandvendorkveten       1
Name: webCampaign, Length: 881, dtype: int64

In [153]:
campaigns

{'Brand',
 'DSA - produkty',
 'Obsah - Dynamický remarketing',
 'Obsah - Remarketing - Akce',
 'PLA',
 'Top produkty'}

In [151]:
set(pokus['webCampaign'].value_counts().index)

{114693,
 114696,
 57364,
 114708,
 114711,
 65569,
 98345,
 73772,
 122925,
 57416,
 81997,
 32851,
 90195,
 8280,
 24673,
 106594,
 16484,
 114789,
 24679,
 131188,
 24694,
 114816,
 73874,
 98450,
 82069,
 49306,
 123042,
 73891,
 8360,
 131242,
 65717,
 90297,
 90307,
 114889,
 131273,
 98507,
 106702,
 131279,
 98512,
 131280,
 73939,
 90345,
 65770,
 24817,
 82161,
 114935,
 98555,
 8451,
 106756,
 114949,
 123145,
 65826,
 24867,
 24868,
 98597,
 65834,
 123179,
 74032,
 33078,
 90424,
 65879,
 74076,
 106844,
 115038,
 57695,
 41314,
 359,
 106866,
 16762,
 82301,
 123268,
 123277,
 106908,
 115100,
 16798,
 115103,
 98721,
 421,
 426,
 49580,
 123313,
 115122,
 49587,
 25017,
 74171,
 98756,
 57799,
 115146,
 98765,
 106957,
 66001,
 106963,
 16857,
 82402,
 16868,
 115173,
 49639,
 57832,
 123372,
 82415,
 106996,
 106999,
 123385,
 66044,
 518,
 33286,
 33287,
 8716,
 90643,
 549,
 115240,
 16937,
 107049,
 49713,
 8762,
 25146,
 123462,
 33352,
 74320,
 8787,
 66138,
 66140

In [102]:
pst = data[(data['site']=='kytary.sk') & (data['platformCode']=='AdWords')]

In [113]:
pst.iloc[1]['currencyCode']

'CZK'

{'(A) Bicí - Obecná',
 '(A) Brand',
 '(A) DSA (vše)',
 '(A) DSA - kategorie',
 '(A) Dynamický remarketing',
 '(A) Fender - modely',
 '(A) Hudební nástroje',
 '(A) Kategorie + značky',
 '(A) Klasický remarketing - Výprodej',
 '(A) Klávesy - obecná',
 '(A) Klávesy - obecná - EXACT',
 '(A) Kytary - obecná',
 '(A) Kytary - obecná - EXACT',
 '(A) Kytary - struny',
 '(A) Kytary - značky',
 '(A) Mikrofony - Obecná (Royal Berl)',
 '(A) Mikrofony - obecná',
 '(A) Obsah - Dynamický remarketing',
 '(A) Obsah - Remarketing - Akce',
 '(A) Obsah - Remarketing klasický',
 '(A) PLA',
 '(A) PLA - obecná',
 '(A) PLA - produkty - PNO 1 + 4 %',
 '(A) PLA - produkty - PNO 7 + 10 %',
 '(A) PLA - výrobce',
 '(A) PLA - výrobce - přesná',
 '(A) SDC - Smart Display Campaign',
 '(A) Sluchátka - Obecná (Royal Berl)',
 '(A) Sluchátka - obecná',
 '(A) Top produkty',
 '(A) Ukulele',
 '(CZ) Baskytary - obecná',
 '(CZ) Baskytary - obecná + značky',
 '(CZ) Baskytary - pouzdra',
 '(CZ) Bicí - obecná',
 '(CZ) Bicí - obec

In [75]:
bb

,platformCode,webCampaign,visitsBeforeTransformations,webCampaignBeforeTransformation
0,(not tracked),(not tracked),0,461
1,AdWords,(A) Bicí - Obecná,205,83
2,AdWords,(A) Brand,4947,295
3,AdWords,(A) DSA (vše),3257,604
4,AdWords,(A) DSA - kategorie,513,167
...,...,...,...,...
739,Yottly,yot_trigger_cart_kosik,4,3
740,Yottly,yot_trigger_fmcg_automat_palicky_obecne,1,1
741,Yottly,yot_trigger_fmcg_automat_struny_kytary,7,5
742,Yottly,yot_trigger_zsv_intention_default,74,45


In [9]:
not_platform = ['(direct)', '(not tracked)']

def get_platforms(data):
    return data['platformCode'].value_counts().to_frame().to_dict()['platformCode']

def get_campaigns(data, platform, which='web'):
    if which not in ['web', 'ad']:
        raise KeyError('No such campaign')
    
    if which == 'web':
        col_name = 'webCampaign'
        numeric = 'visits'
        
    if which == 'ad':
        col_name = 'adCampaign'
        numeric = 'clicks'
    
    
    a = data.groupby(['platformCode', 'webCampaign']).agg({numeric:sum})
    b = data.groupby(['platformCode'])['webCampaign'].value_counts().to_frame()
    bb = pd.concat([a, b], axis=1, sort=False)
    bb.rename(columns={'visits': 'visitsBeforeTransformations', 'webCampaign':'webCampaignBeforeTransformation'})
    
    return bb

def map_campaigns(webC, adC, transforms='no'):
    # webC and adC are dataframes with multiindex
    
    pass





#     data_per_platform = data[data['platformCode']==platform]
    
#     # preparation of data
#     campaigns_num_A = data_per_platform.groupby(col_name).agg({numeric:sum})
#     campaigns_num_B = data_per_platform[col_name].value_counts().to_frame()
#     return pd.concat([campaigns_num_A, campaigns_num_B], axis=1, sort=False)
    
def initial_transforms(df):
    # apply transformations
    # float.nan to empty

    
    campaigns_proc.index = campaigns_proc.index.fillna('')
    if len(campaigns_proc) == 0:
        return campaigns_proc
        
    # lowercase
    campaigns_proc.index = campaigns_proc.index.str.lower()
    
    # change plus by space
    campaigns_proc.index = campaigns_proc.index.str.replace('+', ' ')
    
    # delete diacritics
    campaigns_proc.index = campaigns_proc.index.map(unidecode.unidecode)
    return df
    
def transform():
    campaigns_num = data_per_platform.groupby(col_name).agg({numeric:sum})
    campaigns_proc = (campaigns_num/campaigns_num.sum(axis=0))*100
        
    # apply transformations
    # float.nan to empty
    campaigns_proc.index = campaigns_proc.index.fillna('')
    if len(campaigns_proc) == 0:
        return campaigns_proc
        
    # lowercase
    campaigns_proc.index = campaigns_proc.index.str.lower()
    
    # change plus by space
    campaigns_proc.index = campaigns_proc.index.str.replace('+', ' ')
    
    # delete diacritics
    campaigns_proc.index = campaigns_proc.index.map(unidecode.unidecode)
    
    
    # check for prefix
    return campaigns_proc





In [3]:
#os.chdir("./ALL_data_Oct_2020/")
files = [file for file in glob.glob(f"{path},*.csv")]
    #print(file)
#     with zipfile.ZipFile(file, 'r') as zip_ref:
#         zip_ref.extractall('./')

In [238]:
v

In [16]:
pls = get_platforms(data)


{'AdWords': 101090,
 'HeurekaCz': 5301,
 'HeurekaSk': 3387,
 'Sklik': 3281,
 'DogNet': 1915,
 'DoubleClick': 1508,
 'Organic': 1339,
 'Arukereso': 1062,
 'Internal': 1005,
 'FacebookBusinessAdsManager': 981,
 'Samba': 930,
 'Compari': 615,
 '(not tracked)': 461,
 'Idealo': 249,
 'Silverpop': 227,
 'Yottly': 208,
 'Ceneo': 198,
 'CjAffiliate': 176,
 'Adminis': 160,
 '(direct)': 145,
 'Adform': 106,
 'ZboziCz': 57,
 'Pricespy': 17,
 'OneSignal': 1}

In [14]:
no_platform = ['(direct)', '(not tracked)']

,platformCode
AdWords,101090
HeurekaCz,5301
HeurekaSk,3387
Sklik,3281
DogNet,1915
DoubleClick,1508
Organic,1339
Arukereso,1062
Internal,1005
FacebookBusinessAdsManager,981


In [15]:
len(data1)

3770

In [16]:
data_new = pd.concat([data, data1])

In [17]:
len(data_new)

135086

In [19]:
data_new.columns

Index(['date', 'site', 'currencyCode', 'dataPartitionCode', 'periodCode',
       'attributionModel', 'dataSourceCode', 'conversionType', 'salesChannel',
       'conversions', 'netRevenue', 'grossProfit', 'deliveries',
       'timeGranularity', 'visits', 'entrances', 'bounces', 'hits',
       'pageViews', 'users', 'newUsers', 'impressions', 'marketingInvestment',
       'clicks', 'platformConversions', 'platformNetRevenue', 'webSource',
       'webMedium', 'webCampaign', 'webAdGroup', 'platformCode',
       'platformName', 'adCampaign', 'adGroup', 'sourcePlatform',
       'deviceCategory', 'channelCode', 'channelName', 'investmentType',
       'periodStartDate'],
      dtype='object')

In [307]:
import json
def add_data_to_dict(output_dict, file_name, platform, not_paired, save=True):
    if file_name in output_dict.keys():
        output_dict[file_name][platform] = not_paired
    else: 
        output_dict[file_name] = {}
        output_dict[file_name][platform] = not_paired
    
    if save:
        with open('analyzed.json', 'w+') as f:
            json.dump(output_dict, f)
            
    return output_dict

In [607]:
files

['aliaticba_Oct_2020.csv',
 'aliaticbg_Oct_2020.csv',
 'aliaticcs_Oct_2020.csv',
 'aliatices_Oct_2020.csv',
 'aliaticgr_Oct_2020.csv',
 'aliatichr_Oct_2020.csv',
 'aliatichu_Oct_2020.csv',
 'aliaticlt_Oct_2020.csv',
 'aliaticpl_Oct_2020.csv',
 'aliaticro_Oct_2020.csv',
 'aliaticrs_Oct_2020.csv',
 'aliaticsi_Oct_2020.csv',
 'aliaticsk_Oct_2020.csv',
 'annarbor_Oct_2020.csv',
 'apotek_Oct_2020.csv',
 'appdownloads_Oct_2020.csv',
 'appleton_Oct_2020.csv',
 'astoreosk_Oct_2020.csv',
 'astoreo_Oct_2020.csv',
 'axa-assistancecz_Oct_2020.csv',
 'axa-assistancehu_Oct_2020.csv',
 'axa-assistancepl_Oct_2020.csv',
 'axa-assistancesk_Oct_2020.csv',
 'bezvasportcz_Oct_2020.csv',
 'bezvasportsk_Oct_2020.csv',
 'blancheportecz_Oct_2020.csv',
 'blancheportesk_Oct_2020.csv',
 'bluestyle_Oct_2020.csv',
 'brwpl_Oct_2020.csv',
 'cocochocosk_Oct_2020.csv',
 'coolstuffcom_Oct_2020.csv',
 'coolstuffde_Oct_2020.csv',
 'coolstuffdk_Oct_2020.csv',
 'coolstufffi_Oct_2020.csv',
 'coolstuffno_Oct_2020.csv',
 'cool

In [1048]:
data1.columns

Index(['Unnamed: 0', 'site', 'currencyCode', 'dataPartitionCode', 'periodCode',
       'conversions', 'netRevenue', 'grossProfit', 'deliveries',
       'timeGranularity', 'visits', 'entrances', 'bounces', 'hits',
       'pageViews', 'users', 'newUsers', 'impressions', 'marketingInvestment',
       'clicks', 'platformConversions', 'platformNetRevenue', 'webSource',
       'webMedium', 'webCampaign', 'deviceCategory', 'platformCode',
       'platformName', 'adCampaign', 'adGroup', 'sourcePlatform',
       'channelCode', 'channelName', 'investmentType', 'periodStartDate',
       'appliedAttributionModel'],
      dtype='object')

In [1037]:
# GET DATA
fl = 34
pl = 3

file_name = files[fl]
data1 = pd.read_csv(file_name)
data1 = delete_zero_rows(data1)
platforms = get_platforms(data1)
print(file_name)
platforms

coolstuffno_Oct_2020.csv


,platformCode
AdWords,4179
Adtraction,2171
FacebookBusinessAdsManager,1536
Bing,1149
E-mail_Newsletter,975
Criteo,160
Prisguiden,159
(not tracked),31
Kelkoo,27


In [ ]:
bad_platforms = ['(not tracked)']

In [1038]:
#platforms = ['FacebookBusinessAdsManager', 'AdWords', 'Bing']
#platforms = ['HeurekaCz', 'Sklik', 'ZboziCz', 'FacebookBusinessAdsManager', 'AdWords']
#platforms = ['AdWords']
#platforms = ['Biano', 'FacebookBusinessAdsManager', 'AdWords', 'Email', 'Favi', 'HeurekaSk']
#platforms = ['AdWords', 'Sklik', 'Email', 'Glami', 'Favi', 'FacebookBusinessAdsManager']
#platforms = ['AdWords', 'FacebookBusinessAdsManager', 'Tradedoubler']
platforms = ['AdWords','FacebookBusinessAdsManager',  'Adtraction', 'Bing']
platform = platforms[pl]
file_name

'coolstuffno_Oct_2020.csv'

In [1039]:
# ANALYZA
webC = get_campaigns(data1, platform, which='web')
webC

,visits
webCampaign,
- holidays | generic | fathers day,0.111111
1. pc - brand,0.111111
1. pc -jul,0.111111
1. pc -tagger og tema,0.111111
aaa - products - cpc,0.555556
p - sok - brand,1.111111
p - sok - generisk - bmm,2.000000
p - sok - generisk - exact,5.333333
p - sok - generisk - gavor - bmm,1.555556


In [1040]:
adC = get_campaigns(data1, platform, which='ad')
adC

,clicks
adCampaign,
- holidays | generic | fathers day,0.166528
aaa - products - cpc,0.749376
green friday,0.000000
p - sok - brand,2.414654
p - sok - generisk - bmm,1.748543
p - sok - generisk - exact,4.995837
p - sok - generisk - gavor - bmm,1.415487
p - sok - generisk - gavor - exact,7.577019
p - sok - generisk - rlsa - broad,0.000000


In [1045]:
note = ''

In [1042]:
# NAPAROVANO
# web -> ad
wa = list(set(webC.index) - set(adC.index))
if len(wa) != 0:
    print('Percentage of not mapped visits ', webC.loc[wa].sum(axis=0))
    not_found_webC = list(wa)
    not_found_webC_perc = webC.loc[wa].sum(axis=0)['visits']
    print(wa)
else: 
    print('all webCampaigns are mapped to adCampaigns')
    not_found_webC = []
    not_found_webC_perc = 0
# ad -> web
aw = list(set(adC.index) - set(webC.index))
if len(aw) != 0:
    print('Percentage of not mapped clicks ', adC.loc[aw].sum(axis=0))
    not_found_adC = list(aw)
    not_found_adC_perc = adC.loc[wa].sum(axis=0)['clicks']
    print(aw)
else:
    print('all adCampaigns are mapped to webCampaigns')
    not_found_adC = []
    not_found_adC_perc = 0



Percentage of not mapped visits  visits    0.333333
dtype: float64
['1. pc -tagger og tema', '1. pc - brand', '1. pc -jul']
Percentage of not mapped clicks  clicks    0.0
dtype: float64


KeyError: "None of [Index(['1. pc -tagger og tema', '1. pc - brand', '1. pc -jul'], dtype='object', name='adCampaign')] are in the [index]"

In [1043]:
# INFORMACE 
# print('Ukladam data pro ', file_name)
row = {'file_name': file_name,
       'platformCode': platform,
       'not_found_webCampaign' : not_found_webC, 
       'visits percentage': not_found_webC_perc,
       'not_found_adCampaign': not_found_adC, 
       'clicks percentage': not_found_adC_perc,
        'note': note}
output = output.append(row, ignore_index=True)
output
# not_paired = {'not paired campaigns': not_found_webC + not_found_adC, 
#               'percentage of clicks/visits': not_found_webC_perc + not_found_adC_perc}

# add_data_to_dict(output_dict, file_name, platform, not_paired, save=True)


,file_name,platformCode,not_found_webCampaign,visits percentage,not_found_adCampaign,clicks percentage,note
0,aliaticba_Oct_2020.csv,FacebookBusinessAdsManager,"[instagram.com, facebook.com, lm.facebook.com,...",6.080186,[],0,
1,aliaticba_Oct_2020.csv,AdWords,"[ba_se_category clothing, ba_se_brands]",0.008024,[],0,
2,aliaticbg_Oct_2020.csv,FacebookBusinessAdsManager,"[aliatic_bg_dpa_prp (new), instagram.com, face...",32.483697,"[h8 - thanh trường, s450 - xem - mt - nhut, s4...",10.4183,
3,aliaticbg_Oct_2020.csv,AdWords,[],0.000000,[],0,
4,aliatices_Oct_2020.csv,FacebookBusinessAdsManager,"[lm.facebook.com, m.facebook.com, es_pe_post, ...",100.000000,[],0,no ad campaigns?! with at least any data
...,...,...,...,...,...,...,...
81,coolstufffi_Oct_2020.csv,Adtraction,"[karolinas kaos, lajipaallikko, naag youtube, ...",100.000000,[-],0,no adCampaign
82,coolstuffno_Oct_2020.csv,AdWords,[],0.000000,[],0,no adCampaign
83,coolstuffno_Oct_2020.csv,FacebookBusinessAdsManager,"[[baileys-julekalender], [baileys], [potegull-...",24.664255,[zalster dpa norway],0,no adCampaign
84,coolstuffno_Oct_2020.csv,Adtraction,"[larsbarfot, det gronne skafferi, kosmetikk-ne...",82.980367,[-],0,no adCampaign


In [1044]:
#output.loc[46, 'note']='no matches'

In [1069]:
output[output['platformCode']=='Favi']

,file_name,platformCode,not_found_webCampaign,visits percentage,not_found_adCampaign,clicks percentage,note
39,astoreosk_Oct_2020.csv,Favi,"[favi-podsedaky-na-stolicky, favi-uteraky-a-os...",100.0,[-],100,no adCampaign
45,astoreo_Oct_2020.csv,Favi,"[favi-zavesy-do-oken, favi-koupelnove-skrinky,...",100.0,[-],100,no adCampaign
56,blancheportecz_Oct_2020.csv,Favi,"[favi-sedaci-vaky-a-pytle, favi-zavesy-do-oken...",100.0,"[mergado feeds, -]",100,no adCampaings


In [1051]:
output['platformCode'].unique()

array(['FacebookBusinessAdsManager', 'AdWords', 'Email', 'HeurekaCz',
       'Sklik', 'ZboziCz', 'Bing', 'Biano', 'Favi', 'HeurekaSk', 'Seznam',
       'Glami', 'Sdovolena', 'FirmyCz', 'exponea', 'Tradedoubler',
       'PartnerAds', 'Adtraction'], dtype=object)

In [267]:
#columns_in_row = ['file_name', 'platformCode', 'not_found_webCampaign', 'visits percentage', 'not_found_adCampaign', 'clicks percentage']
#output = pd.DataFrame(columns=columns_in_row)

TypeError: append() got an unexpected keyword argument 'inplace'

In [201]:
not_found_webC + not_found_adC

['ba_se_brands', 'ba_se_category+clothing']

In [187]:
not_found_webC

['ba_se_brands', 'ba_se_category+clothing']

In [107]:

aa = list(set(webC.index) - set(adC.index))

In [117]:
webC.loc[aa].sum(axis=0)

visits    6.080186
dtype: float64

In [87]:
aa = data1[data1['platformCode']=='FacebookBusinessAdsManager'].groupby('adCampaign').agg({'clicks':sum})

In [89]:
aa.index = aa.index.str.lower()

In [90]:
aa

,clicks
adCampaign,
aliatic_ba_conv,37551
aliatic_ba_dpa_prp,73234
aliatic_ba_dpa_rtg,35383


In [47]:
webC = get_web_campaigns(data1, platforms[0])
adC = get_ad_campaigns(data1, platforms[0])

In [58]:

adC = [i.lower() for i in adC if not isinstance(i, float)]


In [63]:
webC

['ba_discovery',
 'ba_dynamic_remarketing',
 'ba_se_aliatic',
 'ba_se_brands',
 'ba_se_category+clothing']

In [61]:
set(webC) - set(adC)

{'ba_se_brands', 'ba_se_category+clothing'}

In [62]:
set(adC) - set(webC)

set()

In [60]:
adC

['ba_dynamic_remarketing', 'ba_se_aliatic', 'ba_discovery']

In [27]:
a

[1, 2, 4, 6]